# ■Scrapping
- 긁어오고 싶은 부분만큼만 긁어올 수 있어요 (크롤러에이전트보다는 '파서'가 더욱 더 중요해요)
- 특정 사이트 하나 들어갈거고, 우리가 만든 '스크래퍼'가 스크래핑을 하는데 '내가 원하는 xml/text/숫자/이미지'만 빼서 저장하는 구조.
- 크롤링의 일부분이라고 생각 해볼 수도 있어요.


# ■Crawlng
- 통으로 긁어와서 앵커페이지를 찾는 식
- 크롤러 에이전트가 필요해요 (어제 a라는 링크 타고 계속해서 네트워크 돌아다니는 것)
- 인덱싱을 통해서 저장합니다.
        - 페이지 링크를 통해 가중치 매기는거 이해만 하면 될거에요
        - 이전의 것들은 빅데이터 '수집' 측면에서 어떤게 있는지에 대한 전반적인 이야기였던거였죠.
        - 아마 오늘부터 하는 파트가 이전 것들보다 더 도움이 될 거에요.

## ■스크래핑의 Architecture
- 멀티 스레드로 돌면서 인터넷으로 부터 웹페이지 다운받고 (url로)
- 스케쥴러에서 씨드 값을 주면 (스케쥴러 : 어제의 큐 역할) 긁어오고 가져오고 하는 식


### ■ 예제 사이트
- https://validator.w3.org/unicorn/?ucn_lang=ko
- 우리나라 10대 사이트면서 거지같이 만든 '뽐뿌'를 긁어볼거에요:)
    - www.ppomppu.com
         - 가격 비교 사이트래요.
         - 제목, 추천수, 조회수를 긁을거에요
         - focused crawling으로서 여기 있는 데이터만 긁으라고 시킬거에요
         - 3개짜리 튜플에 리스트로 만들면 될거 같아요

In [4]:
import sqlite3
import re
import requests

#### 1.다운로드함수

In [1]:
#잘정의하면계속쓸 download함수
import requests
import requests
import time
import json
import re
from bs4 import BeautifulSoup
url= "http://www.ppomppu.co.kr/zboard/zboard.php" #url은 상황에 맞게 작성
headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
def download(method, url, param=None, data=None):
#없을 수도 있는 애들에 초기값 설정

    try:
        resp=requests.request(method, url,params=param,data=data, headers=headers)
    #여기서 에러 난다면 어디서? 
    #resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(e.code)
        print(e.reason)

    return resp

#### 2.사이트 긁어오기 //[씨드함수(?)]

In [52]:
url="http://www.ppomppu.co.kr/zboard/zboard.php"
param={"id":"ppomppu", "page":1, "divpage":56}
html=download("get",url,param)
dom=BeautifulSoup(html.text, "lxml")

In [53]:
html.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr" />\n<meta http-equiv="Content-Script-Type" content="text/javascript" />\n<meta http-equiv="Content-Style-Type" content="text/css" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="author" content="PPOMPPU CO.">\n<meta name="description" content="뽐뿌">\n<meta name="keywords" content="">\n\n\n<script async=\'async\' src=\'//www.googletagservices.com/tag/js/gpt.js\'></script>\n<script>var googletag = googletag || {};googletag.cmd = googletag.cmd || [];</script>\n\n<!--\n<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=yes, target-densitydpi=device-dpi" />\n-->\n\n\n<meta property="og:image" content="http://www.ppomppu.co.kr/images/icon_app_20160427.png" />\n\n<meta property="og:site_name" content="뽐뿌" />\n\n<tit

#### 3.원하는 부분 추출(내가 했어~!!!)
- 조회수와 추천수 까지 3개를 튜플형식으로 저장해보는걸 더 해야합니다.

In [55]:
# 제목 추출
[(_.text) for _ in dom.select(".list_title")]

['[G마켓] 햇반컵반 버터장조림비빔밥 216g 6개 (15,900원 /무배)',
 '[G마켓] LG 휘센 제습기 DQ168PBE (368,470원/ 무료배송)',
 '[11번가] LG 공식판매점 디오스 인덕션 전기레인지 BEF3G 3구 (1,490,000/무료)',
 '[위메프] EVA 30CM 마사지 폼롤러 (5900/무료)',
 '[위메프] LG 미니빔 HF60LA FHD 1400루멘 (835,640/무료)',
 '[11번가] 리스테린 그린티 마일드 750mlx2+250mlx2  (13,500/무료배송)',
 '[Gsuite] Gsuite Basic India (3700원(월) / 무료)',
 '[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백',
 '[옥션] 식사에반하다25개입 (26,530원 / 무배)',
 '[티몬] (티몬페이전용)해피머니 10만원권 3장(271,500원/쿠폰적용시 269,900원)',
 '[티몬] 해피머니 10만원권 (90,900원/0원)',
 '[티몬] 1등급 한돈 삼겹살 600g (6,500/무료배송)',
 '[우체국쇼핑] 진안 매콤화끈 중독의맛 뼈없는 직화불닭발 1+1(2팩)(8500/무료) 쿠폰적용시7650',
 '[유튜브] 유튜브프리미엄 인도 ( 2200원(월), 3200원(월)6명 / 무료)',
 '[11번가] 벨킨 스마트폰 전기종 호환 충전케이블(9,900/무배)',
 '[네이버스토어팜] 뉴발란스 잔테v3 (49,800원/무료)',
 '[LG생활건강] 홈스타 뿌리는 곰팡이 (9,900/무료)']

### 아래 포문 옆에 설명 잘 써두었으니 참고★★★
- 교수님 정답
- 아래 <tr><td> 사용하는 방식은 2000년대 초반 방식이나 어쨌든 결과를 가져오기만 하면 되는거 아닙니까:)

In [57]:
for tr in dom.select("tr[class^=list]")[1:]: #tr이 list0, list1이 반복되고 있어서, 이렇게 시작되는 애들 중에서 고르려고 한 듯. 첫줄은 설명 공지사항이라 빼고. 첫줄 클래스는 list_notice 
    td= tr.find_all("td", recursive=False)# recursive=False: tr 바로 밑에있는 자식 td만 건들이겠다는 뜻!!
    print(requests.compat.urljoin(url, td[3].a.img["src"]))# urljoin은 상대주소를 절대주소로 바꾸니까 도메인과 url 합침 **
    print(td[3].select("td[valign=middle] > a")[0].text) #에이가 글에/그림에 있었어
    
    #print(re.findall(".+\((.+?)\)"))
    
    temp=td[5].text.split("-") # 추천(td[5]) 비추천 안 나는애 대하려고. 스플릿해서 길이가 0,1인애 등이 있었어요. 그래서 없는애는 무시함. 현재 (추천-비추천)형식
    print((0,0) if len(temp)<2 else ",".join(temp)) #추천 비추천 고루 있는 애만 긁어오게 했어요. ","join(temp)를 통해 (추천-비추천)을 (추천,비추천)형식으로
    print(td[6].text.strip()) ###우리가 데이터를 가지고 올때 개항문자(\n)및 공백이 있으면 숫자로서 활용하기 힘들 수 있으니 작성!
    print()

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/3/small_323883.jpg?t=20190716153054
[G마켓] 햇반컵반 버터장조림비빔밥 216g 6개 (15,900원 /무배)
(0, 0)
104

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/2/small_323882.jpg?t=20190716152450
[G마켓] LG 휘센 제습기 DQ168PBE (368,470원/ 무료배송)
(0, 0)
354

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/1/small_323881.jpg?t=20190716150504
[11번가] LG 공식판매점 디오스 인덕션 전기레인지 BEF3G 3구 (1,490,000/무료)
0 , 1
837

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/0/small_323880.jpg?t=20190716151334
[셀티바] 다이어트유산균(48%할인/무료배송)
0 , 2
463

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/9/small_323879.jpg?t=20190716145420
[위메프] EVA 30CM 마사지 폼롤러 (5900/무료)
3 , 0
2579

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/8/small_323878.jpg?t=20190716145350
[위메프] LG 미니빔 HF60LA FHD 1400루멘 (835,640/무료)
(0, 0)
1217

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/7/small_323877.jpg?t=20190716144040
[11번가] 리스테린 그린티 마일드 750mlx2+250mlx2  (13,500/무료배송)
(0, 0)
561

http://cd

# DO IT YOUR SELF====================================
## [crawling]
 1. Focused Crawling(뽐뿌-자유게시판-10페이지 내에서)
 2. 각 페이지 당 게시글(제목) 부분의 a 수집
 3. 수집된 a로부터 scraping
 - 10개 페이지만 목록을 가져오고, 앵커가 붙어있는 [글/제목/댓글]수집.   (특정 영역안에 있는 도메인 건들이니까 포커스드)

## [Scraping]
1. 제목, 본문내용, 댓글 파싱(댓글은 읽거나 여러개 있음)
- 스크래핑 잘 안 될 수 있으니 열심히 하길 바람

##### 1. 다운로드 함수 사용=======================================
- 내 시도
- 현재 뽐뿌 사이트 긁어옴

In [2]:
#잘정의하면계속쓸 download함수
import requests
import requests
import time
import json
import re
from bs4 import BeautifulSoup
url= "http://www.ppomppu.co.kr/zboard/zboard.php" #url은 상황에 맞게 작성
headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
def download(method, url, param=None, data=None):
#없을 수도 있는 애들에 초기값 설정

    try:
        resp=requests.request(method, url,params=param,data=data, headers=headers)
    #여기서 에러 난다면 어디서? 
    #resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(e.code)
        print(e.reason)

    return resp

#### 2. 사이트 긁어오기

In [8]:
url="http://www.ppomppu.co.kr/zboard/zboard.php"
while(i<10)
{i=i+1
print(i)}
param={"id":"freeboard", "page":1,"page":2,"page":3,"page":4,"page":5,"page":6,"page":7,"page":8,"page":9,"page":10,"divpage":1221}
html=download("get",url,param)
dom=BeautifulSoup(html.text, "lxml")

SyntaxError: invalid syntax (<ipython-input-8-0051ef1a765b>, line 2)

#### 3. html.text로 확인

In [25]:
html.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr" />\n<meta http-equiv="Content-Script-Type" content="text/javascript" />\n<meta http-equiv="Content-Style-Type" content="text/css" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="author" content="PPOMPPU CO.">\n<meta name="description" content="뽐뿌">\n<meta name="keywords" content="">\n\n\n<script async=\'async\' src=\'//www.googletagservices.com/tag/js/gpt.js\'></script>\n<script>var googletag = googletag || {};googletag.cmd = googletag.cmd || [];</script>\n\n<!--\n<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=yes, target-densitydpi=device-dpi" />\n-->\n\n\n<meta property="og:image" content="http://www.ppomppu.co.kr/images/icon_app_20160427.png" />\n\n<meta property="og:site_name" content="뽐뿌" />\n\n<tit

#### 3. 원하는 부분 추출(크롤링)

In [29]:
for tr in dom.select("tr[class^=list]")[1:]:
    td=tr.find_all("td", recursive=False)
    print( td[2].select("td[align=left]>a")[0].text)

네일동 까페는  미첬네요 ㄷㄷㄷㄷㄷ   완전 친일동입니다
징검다리 날씨네요
경남 중산리 계곡 클라스
점심)쌀국수&볶음밥vs돈까스&냉모밀
환경부에서 제작지원 해주는 드라마.jpg
환율전쟁 3라운드 막 올랐다 트럼프 직접개입 우려 확산
영화 망하면 크게 손해입는다는게 엄복동으로 증명되었죠
오늘 보았던 최고의 댓글
중소기업 구내식당 경험담
불매운동
이 블로그 주인이 제이쓴이었네요
초등학생 필독서에 그딴 거 끼어넣지마
영등포역 노점상 싹 엎어버려서 깔끔해졌네요
미니 무선선풍기 충전하시면서 쓰시나요..
영화기생충
성수 블루보틀 늦게 와봤는데 괜찮네요
직장내 괴롭힘을 금지하는 법이????
구미 이노텍 단기계약직 면접갑니다..
은행들이 변동금리 낮췄다네요
스타벅스 자개 카드 사진
원빈은 앞으로도 영화 안할거같아요
햐.. 저도 동참 하고싶은데. 딱히 할만한게 없네요.
인천에서 떡상하고 있다는 카드.jpg
넷플렉스도 공구하는 분들 터키 돈으로 창조경제 하는거였군여~
이발한날 괜히 피곤한분 계시나요?
일본여행취소건수 현황
근로계약서와 동시에 사직서
홍콩반점 탕짜면
다이어트 한지 이제 거의 한달 �楹六� ㅠ


## Focused 크롤링(do it your self 선생님 코드)★★★★★=============================

## parseURL(seed)

In [3]:
def parseURL(seed):
    html=download("get",seed) #메소드는 '겟'방식이고, seed로 URL 받아요
    dom=BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(seed, _.find_parent()["href"])
           #requests.compat.urljoin(seed, --) 이건 상대주소를 절대 주소로 변경하는 거에요
            for _ in dom.select("font.list_title") if _.find_parent() and _.find_parent().has_attr("href")
           ]

## parseContent(url)

In [4]:
def parseContent(url):
    html=download("get", url)
    dom=BeautifulSoup(html.text, "lxml")
    #그 url 타고 들어가보니 있는 title, body, comments임
    return{
            "title":dom.select_one("font.view_title2").text.strip(),#strp은 양 옆에 있는 공백이나 \n 개행 문자 없애줌
            "body":dom.select_one("td.board-contents").text.strip(),
            "comments":[_.text.strip() for _ in dom.select("div[id^=commentContent_]")]
    }
# 만일 여기서 body 본문 내용이 안 된다면, 뽐뿌를 들어가볼게요
# 개발자모드 F12로 보면 td. boad-contents가 잘 보이는데, 소스코드로 보면 그 클래스 뒤에 han이라는 클래스가 또 있어요
# 그건 처음에 td.board-contents로 클래스를 만들었는데, 나중에 디자인 등의 문제로 인해 han 클래스가 추가된 것이에요
# 그럴 경우는 이 윗단을 부르고, > 안쓰고 한 칸 띄우면 자손 부를 수 있으니까 han 클래스로 해서 부르면 될 겁니다.

#### 크롤링- 선생님 코드

#### 1. 다운로드 함수
- 뽐뿌 사이트 '자유게시판'에서 게시글 10페이지 긁어올거에요

In [11]:
#DOWNLOAD 함수- 성공적으로 다 긁어오기- 자유게시판
import requests
import requests
import time
import json
import re
from bs4 import BeautifulSoup
url= "http://www.ppomppu.co.kr/zboard/zboard.php" #url은 상황에 맞게 작성
headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
def download(method, url, param=None, data=None):
#없을 수도 있는 애들에 초기값 설정

    try:
        resp=requests.request(method, url,params=param,data=data, headers=headers)
    #여기서 에러 난다면 어디서? 
    #resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(e.code)
        print(e.reason)

    return resp

#### 2. 큐 작업

In [12]:
# 다 긁어온 다음에 원하는 부분(현재는 url30개씩있는 페이지 10개, 가져오고 본건 false 처리해서 필터링 하기)
url="http://www.ppomppu.co.kr/zboard/zboard.php"
html=download("get", url, param={"id":"freeboard"})
dom=BeautifulSoup(html.text, "lxml")

seed=list()
seed.extend([requests.compat.urljoin(url, _["href"]) for _ in dom.select("#page_list a")])# 샵은 id, 점은 클래스. 자식은 꺽쇄, 자손은 한칸 뜀 

seen=list()
queue=list()

while seed:
    baseURL=seed.pop(0)
    seen.append(baseURL)
    linkList=parseURL(baseURL)
    
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue:{0}, Seen:{1}".format(len(queue), len(seen)))

Queue:30, Seen:1
Queue:60, Seen:2
Queue:90, Seen:3
Queue:120, Seen:4
Queue:150, Seen:5
Queue:180, Seen:6
Queue:210, Seen:7
Queue:240, Seen:8
Queue:270, Seen:9
Queue:300, Seen:10


### 스크래핑- 선생님 코드(이거 크롤링 속에 있음)

In [16]:
#contents=list()
#while queue:
#    baseURL=queue.pop(0)
#    seen.append(baseURL)
#    content=parseContent(baseURL)
# len(contents)
# contents[0]

< 긁어오는 식 예시>
- for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
- [_["href"] for _ in dom.find_all("a")] 
- [_.find_parent()["href"] for _ in dom.select(".LC20lb")]
- [(_["href"], _.text) for _ in dom.select(".rc > .r a[href^=http]")]
- [_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title")]
- [ (_["href"], _.text) for _ in dom.select("div.news ul.type01>li a._sp_each_title")]
- [ _.text for _ in dom.select("div#newsColl div.cont_inner a.f_link_b")]
- [ (_["href"], _.text) for _ in dom.select("div#newsColl div.cont_inner a.f_link_b")]


- for tr in dom.select("tr[class^=list]")[1:]: #tr이 list0, list1이 반복되고 있어서, 이렇게 시작되는 애들 중에서 고르려고 한 듯. 첫줄은 설명 공지사항이라 빼고 
    td= tr.find_all("td", recursive=False)# recursive=False: tr 바로 밑에있는 자식 td만 건들이겠다는 뜻!!
    print(requests.compat.urljoin(url, td[3].a.img["src"]))# urljoin은 상대주소를 절대주소로 바꾸니까 도메인과 url 합침 **
    print(td[3].select("td[valign=middle] > a")[0].text) #에이가 글에/그림에 있었어
    temp=td[5].text.split("-") # 추천 비추천 안 나는애 대하려고. 스플릿해서 길이가 0,1인애 등이 있었어요. 그래서 없는애는 무시하고
    print((0,0) if len(temp)<2 else ",".join(temp)) #추천 비추천 고루 있는 애만 긁어오게 했어요. 
    print(td[6].text.strip())
    print()




### ■크롤링 선생님 코드에서 DB에 content를 담으려고해
- 테이블에 댓글과 본문을 담으려고 한다면 몇 개를 추가하면 좋을까요?
- 현재 몇 개가 반복되고 있나요? 
    - 최소 2개 필요해요(게시글, 댓글)
- 어제 것에 이어서 해보죠

#### 3.DB 저장
- 테이블 생성

In [6]:
import sqlite3
con= sqlite3.connect("bot.db")
cur=con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS table3;
    CREATE TABLE table3(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        content TEXT NOT NULL, 
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    
    );
""")


#table4 생성
cur.executescript("""
    DROP TABLE IF EXISTS table4;
    CREATE TABLE table4(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table3_id INTEGER NOT NULL,
        comment TEXT NOT NULL
    
    );
""")

#### 4. 테이블 조인
- 위의 과정을 하나로 조인해볼게요

In [7]:
cur.executescript("""
    DROP TABLE IF EXISTS table3;
    CREATE TABLE table3(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        content TEXT NOT NULL, 
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    
    );

    DROP TABLE IF EXISTS table4;
    CREATE TABLE table4(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table3_id INTEGER NOT NULL,
        comment TEXT NOT NULL
    
    );
""")

#### 5. 테이블에 삽입
- insert 해봐요

In [10]:
#contents=list()
while queue:
    baseURL=queue.pop(0)
    seen.append(baseURL)
    content=parseContent(baseURL)#parseContet는 url 넣으면 홈페이지에서 원하는 title, body comment 위치를 갖고오게해
    cur.execute("""
       INSERT INTO table3(title,content) VALUES(?,?)
    """, [content["title"], content["body"]]
    )
    
    con.commit()
    cur.execute(
    """
        SELECT id
        FROM table3
        WHERE title=? AND content=? ;
    """, [content["title"], content["body"]]
    ) #execute는 한 줄씩 실행이래요 그래서 여기는 insert select 부분에 ; 안쓰는데, executescript 경우는 꼭 써야한대. 여러개 넣어서
    
    table3_id=cur.fetchone()[0]#첫번째 줄의 0번째가 id라서 그 값을 저장해요
    
    for comment in content["comments"]:
        cur.execute("""
            INSERT INTO table4 (table3_id, comment) VALUES(?,?)
        """,[table3_id, comment]) #위에서는 select id로 table3에 있는거 빼내서 table3_id에 저장했고, 이걸 이제 talbe4에 넣네
    
    con.commit()

In [ ]:
### 교수님은 게시글 300개 댓글 2천여개 수집하셨대